In [ ]:
pip install transformers

In [ ]:
pip install torch

In [5]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from tqdm import tqdm

# Define the path to your CSV file
csv_file = '/content/AWS Project Dataset.csv'

# Define the number of training epochs and batch size
epochs = 10
batch_size = 4
accumulation_steps = 8  # Number of steps before backward pass and parameter update

# Load the data from the CSV file
data = pd.read_csv(csv_file, encoding='latin-1')

# Preprocessing
data = data.dropna()  # Drop rows with missing values
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row['Question']
        answer = row['Answer']

        # Tokenize the input text
        inputs = self.tokenizer.encode_plus(
            question,
            answer,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        return input_ids, attention_mask

# Create an instance of the custom dataset
dataset = CustomDataset(data, tokenizer)

# Create a data loader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

accumulated_loss = 0

for epoch in range(epochs):
    model.train()

    for i, batch in tqdm(enumerate(dataloader), desc=f'Epoch {epoch + 1}/{epochs}'):
        input_ids, attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=None)
        loss = outputs[0]

        # Calculate mean loss across the batch
        loss = loss.mean()

        loss.backward()

        accumulated_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    average_loss = accumulated_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{epochs} - Average Loss: {average_loss:.4f}')
    accumulated_loss = 0

# Save the trained model
model.save_pretrained('trained_model')
tokenizer.save_pretrained('trained_model')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1/10 - Average Loss: -0.2308


Epoch 2/10: 4it [01:49, 27.41s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 2/10: 11it [05:03, 27.56s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 2/10: 14it [06:25, 27.40s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 2/10: 17it [07:48, 27.63s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be em

Epoch 2/10 - Average Loss: -0.8369


Epoch 3/10: 1it [00:25, 25.73s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 3/10: 3it [01:22, 27.77s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 3/10: 4it [01:48, 27.08s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 3/10: 14it [06:26, 27.83s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empt

...

Epoch 9/10: 3it [01:22, 27.63s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 9/10: 4it [01:48, 27.04s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 9/10: 14it [06:26, 27.82s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 9/10: 21it [09:40, 27.62s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be emp

Epoch 9/10 - Average Loss: -4.7837


Epoch 10/10: 2it [00:57, 28.81s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 10/10: 4it [01:53, 28.33s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 10/10: 6it [02:49, 28.12s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 10/10: 18it [08:24, 27.81s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be 

Epoch 10/10 - Average Loss: -5.2776


('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/vocab.txt',
 'trained_model/added_tokens.json')

In [21]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the trained model and tokenizer
model_path = "/content/trained_model/trained_model"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Prediction
def predict_label(model, tokenizer, input_text):
    # Tokenize the input text
    inputs = tokenizer.encode_plus(input_text, truncation=True, padding=True, return_tensors="pt")

    # Forward pass
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

    return predicted_label

# Example prediction
question = "What is the student visa procedure?"
predicted_label = predict_label(model, tokenizer, question)
print("Predicted Label:", predicted_label)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


OSError: ignored